In [2]:
# https://python.langchain.com/docs/how_to/custom_embeddings/

from typing import List, Iterable, Union

from langchain_core.embeddings import Embeddings

import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig

def _last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

class MyEmbeddings(Embeddings):
    def __init__(
        self,
        model: str = "Qwen/Qwen3-Embedding-8B",
        *,
        max_length: int = 8192,
        device = None
    ):
        self.model_name = model
        self.max_length = max_length

        self._tokenizer = tokenizer = AutoTokenizer.from_pretrained(self.model_name, padding_side='left')

        bnb_config = BitsAndBytesConfig(load_in_4bit=True)
        self._model = AutoModel.from_pretrained(self.model_name,quantization_config=bnb_config,).eval()

        if device is None:
            if torch.cuda.is_available():
                device = "cuda"
            elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
                device = "mps"
            else:
                device = "cpu"
        self._device = torch.device(device)
        self._model.to(self._device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self._embed_texts(texts)

    def embed_query(self, text: str) -> List[float]:
        return self._embed_texts([text])[0]

    @torch.inference_mode()
    def _embed_texts(self, texts: Union[List[str], Iterable[str]]) -> List[List[float]]:
        batch_dict = self._tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        batch_dict.to(self._device)
        outputs = self._model(**batch_dict)
        

        embeddings = _last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        out = embeddings.tolist()
        return out

In [3]:
from langchain_chroma import Chroma

embeddings = MyEmbeddings()

DB_PATH = "./data/embedded/chroma_db"

persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="samsung_quarterly_report",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
retriever = persist_db.as_retriever()

In [5]:
# from dotenv import load_dotenv

# load_dotenv()
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "55기 주주총회에 대해 알려줘"
response = chain.invoke(question)
print(response)

제55기 정기주주총회는 2024년 3월 20일에 개최되었습니다. 주요 안건으로는 제55기 재무상태표, 손익계산서 및 이익잉여금처분계산서 등 재무제표 승인, 사외이사 신재윤 선임, 감사위원회 위원이 되는 사외이사 조혜정 선임, 감사위원회 위원 유명희 선임, 이사 보수한도 승인, 정관 일부 변경 등이 있었습니다. 모든 안건은 가결되었습니다. 주주총회에서는 55기 영업성과 및 배당 관련 설명, 주주가치 제고 방안, 사업경쟁력 관련 질의응답 등이 논의되었습니다.


In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
chain.invoke({"question": question, "context": context})

"This morning's meeting was highly productive, as we successfully addressed and resolved all world conflicts."